# 目次
1. コレスキー分解
2. 線型方程式
3. lapack

# 0. 準備

In [2]:
import scipy 
import numpy as np
import pandas as pd
import pods 

In [3]:
from scipy.linalg import cholesky, solve_triangular, LinAlgError

In [4]:
from scipy.linalg.lapack import dpotrf, dtrtri, dpotrs, dpotri

In [84]:
class RBF:
    def __init__(self, variance=1., lengthscale=0.1):
        self.variance=variance
        self.lengthscale=lengthscale
        # self.r = self._euclidean_distance
        
    def K(self, X, X2=None):
        return self.variance * np.exp(-0.5 * (self._euc_dist(X, X2) / self.lengthscale)**2)
        # return self._euc_dist(X, X2)
        
    def _euc_dist(self, X, X2):
        if X2 is None:
            # print("X2 is None")
            # print(X2)
            Xsq = np.sum(np.square(X),1)
            r2 = -2.*(np.dot(X, X.T)) + (Xsq[:,None] + Xsq[None,:]) 
            r2 = np.clip(r2, 0, np.inf)
            np.fill_diagonal(r2, 0.)
            return np.sqrt(r2)
        else:
            # print(X)
            # print(X2)
            X1sq = np.sum(np.square(X),1)
            X2sq = np.sum(np.square(X2),1)
            r2 = -2.*np.dot(X, X2.T) + (X1sq[:,None] + X2sq[None,:])
            r2 = np.clip(r2, 0, np.inf)
            return np.sqrt(r2)


def generate_non_pd_mat():    
    # Create PD matrix
    A = np.random.randn(20, 100)
    A = A.dot(A.T)
    # Compute Eigdecomp
    vals, vectors = np.linalg.eig(A)
    # Set smallest eigenval to be negative with 5 rounds worth of jitter
    vals[vals.argmin()] = 0
    default_jitter = 1e-6 * np.mean(vals)
    vals[vals.argmin()] = -default_jitter * (10 ** 3.5)
    A_corrupt = (vectors * vals).dot(vectors.T)
    return A_corrupt


def custom_cholesky(A, max_tries=5):
    A = np.ascontiguousarray(A) # パフォーマンス向上 計算結果にも影響
    diag_A = np.diag(A)
    jitter = diag_A.mean() * 1e-6
    num_tries = 0
    
    try:
        L = cholesky(A, lower=True)
        return L
    except LinAlgError:
        num_tries += 1
        
    while num_tries <= max_tries and np.isfinite(jitter):
        try:
            L = cholesky(A + np.eye(A.shape[0]) * jitter, lower=True)
            return L
        except LinAlgError:
            jitter *= 10
            num_tries += 1
    
    raise LinAlgError("Matrix is not positive definite, even with jitter.")


def symmetrify_matrix(A, upper=False):
    triu = np.triu_indices_from(A,k=1)
    if upper:
        A.T[triu] = A[triu]
    else:
        A[triu] = A.T[triu]
    return A

In [6]:
np.random.seed(seed=0)
data = pods.datasets.olympic_100m_men()
X, Y = data["X"], data["Y"]
X_pred = np.linspace(X[:,0].min() - 30,
                     X[:,0].max() + 30,
                     500).reshape(-1,1)

# 1. コレスキー分解と jitter

In [7]:
A = generate_non_pd_mat()
print(A.shape)
display(pd.DataFrame(A).head())

(20, 20)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,100.694709,13.393004,6.856588,-9.335437,-2.798888,-3.200455,-13.350150,-0.322746,5.952898,-1.738560,7.341113,-20.082348,-2.620093,-14.020931,-0.002915,-17.326635,-12.785996,-21.552479,15.561366,-16.946348
1,13.393004,106.469565,5.126617,8.365238,-4.577476,-12.119513,-13.638746,8.962440,0.733034,-11.309637,10.379592,-13.374124,0.204128,-1.755680,-1.127009,-3.383279,8.738828,-4.390738,-2.898097,15.654230
2,6.856588,5.126617,90.974871,12.176315,9.826938,19.045201,5.775225,-5.556997,-7.209078,-13.620808,-5.013334,9.315395,3.086520,3.052010,1.253471,-3.626596,-12.044544,5.511489,-1.821928,0.972266
3,-9.335437,8.365238,12.176315,85.027530,-7.020712,31.470890,23.208997,-20.540859,5.189772,1.355420,-0.157656,-13.895595,2.149917,-6.594023,10.963412,-2.145887,-9.722722,3.209814,-9.039136,0.290485
4,-2.798888,-4.577476,9.826938,-7.020712,107.385855,9.459703,11.339283,-4.715181,-5.608686,-3.933285,12.814422,28.178581,12.119413,-5.248214,-16.061067,16.913818,12.789264,3.533077,-9.918885,-9.794405


In [18]:
scipy.linalg.cholesky(A)

LinAlgError: 20-th leading minor of the array is not positive definite

GPy の jitchol 関数と同じ処理

なぜ対角成分の平均値を用いて jitter の初期値を設定することについての正当性は今の所わからない

In [19]:
def custom_cholesky(A, max_tries=5):
    A = np.ascontiguousarray(A) 
    diag_A = np.diag(A)
    jitter = diag_A.mean() * 1e-6 
    num_tries = 0
    
    try:
        L = cholesky(A, lower=True)
        return L
    except LinAlgError:
        num_tries += 1
        
    while num_tries <= max_tries and np.isfinite(jitter):
        try:
            L = cholesky(A + np.eye(A.shape[0]) * jitter, lower=True)
            return L
        except LinAlgError:
            jitter *= 10
            num_tries += 1
    
    raise LinAlgError("Matrix is not positive definite, even with jitter.")

In [21]:
display(pd.DataFrame(custom_cholesky(A)))

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,10.079155,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-1.272912,10.020976,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.444715,1.764754,9.162978,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,-1.924499,0.178279,-1.747797,8.829205,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,-1.764383,-1.654793,-0.383877,1.330341,8.937070,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.566955,0.372367,-0.074868,0.416500,-2.110996,9.399257,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,-0.188470,0.593867,-0.226173,-1.059401,-0.802761,-0.603348,9.274701,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,1.097937,-0.270480,0.660578,1.894447,-1.120371,1.710281,-0.355114,8.792784,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,0.250526,0.705124,0.504488,-0.125013,1.315763,0.273342,0.859833,1.113350,9.239257,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,-0.749856,0.226164,1.326137,-0.286639,-0.549091,0.013889,0.185065,-0.903127,-0.985819,8.762896,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


# 2. LAPACK (Linear Algebra Package)

LAPACK は FORTRAN の線形代数の数値計算用ライブラリで scipy からも呼び出して使用できる．

丸め誤差等の問題から数式上では同等でも計算結果が微妙に異なる場合があるのでそれを実際に確認したい．

<!-- ## 2.1 ~~Is $A \backslash y == L^\top \backslash (L \backslash y)$ ?~~ -->
## 2.1 線型方程式を解く

ある $n \times n$ の実正定値対称行列 $A$ と $n \times m$ の行列 $y$ に対して

$$
A x = y
$$

を満たすような $n\times m$ の行列 $x$ を求めたい．

ここで $A = LL^\top$ のようにコレスキー分解をすると $x$ は

$$
x = L^\top \backslash (L \backslash y)
$$

のようにして計算できる． 

LAPACK には `DPOTRS` という上記の線型方程式 $Ax = y$ を解く関数が存在する．
ただし，引数の１つには $A$ そのものではなく $L$ を渡す点には注意が必要であり，結局のところ `scipy.linalg.solve_triangular` を２度適用するのと同等であることが確認できる．

In [21]:
A = generate_non_pd_mat()
b = np.random.rand(A.shape[0],1)
# c = np.eye(A.shape[0])

L = custom_cholesky(A)

In [26]:
A_y, _ = dpotrs(L, b, lower=True) # 引数は三角行列を与える
LT_L_y = solve_triangular(L.T, solve_triangular(L, b, lower=True))

np.unique(A_y == LT_L_y)

array([ True])

## 2.2 逆行列を求める

実正定値対称行列 $A$ の逆行列を以下の4パターンで求めて比較する．

1. `DPOTRI` を使用する

2. 線型方程式を解く
$$
A^{-1} = A \backslash I
$$

1. コレスキー分解を利用する
$$
A^{-1} = L^{-\top}L^{-1}
$$

2 でもコレスキー分解は利用している．

In [79]:
# 正定値行列の生成
A = np.random.randn(20, 100)
A = A @ A.T

L = custom_cholesky(A)

In [80]:
A_inv, info = dpotri(L, lower=True)
print(info) # 0 が返却されたら成功
display(pd.DataFrame(A_inv).head())

0


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.012248,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000140,0.012355,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.001994,0.000723,0.012951,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.001304,0.001528,-0.000053,0.012543,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.001945,0.001794,-0.000852,0.001020,0.014525,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
A_inv = symmetrify_matrix(A_inv)
display(pd.DataFrame(A_inv).head())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.012248,0.000140,-0.001994,0.001304,0.001945,-0.001190,-0.000150,-0.001785,0.001686,0.000066,0.001683,-0.000680,-0.002132,-0.000263,0.001831,-0.002084,0.001304,-0.000197,0.000847,0.001176
1,0.000140,0.012355,0.000723,0.001528,0.001794,0.001394,0.000892,-0.002599,0.001468,-0.001099,0.001164,0.000180,0.001735,0.000687,0.000266,-0.000304,0.001885,-0.002428,-0.001702,0.001074
2,-0.001994,0.000723,0.012951,-0.000053,-0.000852,0.001067,0.001231,0.001516,0.000820,0.000027,-0.001144,0.000260,0.000917,0.001398,0.000488,0.002026,0.000368,0.002173,0.000700,-0.000374
3,0.001304,0.001528,-0.000053,0.012543,0.001020,0.000961,0.001045,-0.000435,0.002855,0.000812,0.002634,-0.000612,0.000330,0.001432,0.000442,0.000063,0.002760,0.000948,-0.000924,0.002184
4,0.001945,0.001794,-0.000852,0.001020,0.014525,-0.001656,0.001454,-0.001324,-0.000028,-0.000170,0.000529,0.000088,-0.000122,-0.000097,-0.000385,-0.000529,-0.000042,0.000889,-0.000345,-0.000043


In [81]:
A_inv_2 = solve_triangular(L.T, solve_triangular(L, np.eye(A.shape[0]), lower=True))
display(pd.DataFrame(A_inv_2).head())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.012248,0.000140,-0.001994,0.001304,0.001945,-0.001190,-0.000150,-0.001785,0.001686,0.000066,0.001683,-0.000680,-0.002132,-0.000263,0.001831,-0.002084,0.001304,-0.000197,0.000847,0.001176
1,0.000140,0.012355,0.000723,0.001528,0.001794,0.001394,0.000892,-0.002599,0.001468,-0.001099,0.001164,0.000180,0.001735,0.000687,0.000266,-0.000304,0.001885,-0.002428,-0.001702,0.001074
2,-0.001994,0.000723,0.012951,-0.000053,-0.000852,0.001067,0.001231,0.001516,0.000820,0.000027,-0.001144,0.000260,0.000917,0.001398,0.000488,0.002026,0.000368,0.002173,0.000700,-0.000374
3,0.001304,0.001528,-0.000053,0.012543,0.001020,0.000961,0.001045,-0.000435,0.002855,0.000812,0.002634,-0.000612,0.000330,0.001432,0.000442,0.000063,0.002760,0.000948,-0.000924,0.002184
4,0.001945,0.001794,-0.000852,0.001020,0.014525,-0.001656,0.001454,-0.001324,-0.000028,-0.000170,0.000529,0.000088,-0.000122,-0.000097,-0.000385,-0.000529,-0.000042,0.000889,-0.000345,-0.000043


In [82]:
A_inv_3 = np.linalg.pinv(A)
display(pd.DataFrame(A_inv_3).head())

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.012248,0.000140,-0.001994,0.001304,0.001945,-0.001190,-0.000150,-0.001785,0.001686,0.000066,0.001683,-0.000680,-0.002132,-0.000263,0.001831,-0.002084,0.001304,-0.000197,0.000847,0.001176
1,0.000140,0.012355,0.000723,0.001528,0.001794,0.001394,0.000892,-0.002599,0.001468,-0.001099,0.001164,0.000180,0.001735,0.000687,0.000266,-0.000304,0.001885,-0.002428,-0.001702,0.001074
2,-0.001994,0.000723,0.012951,-0.000053,-0.000852,0.001067,0.001231,0.001516,0.000820,0.000027,-0.001144,0.000260,0.000917,0.001398,0.000488,0.002026,0.000368,0.002173,0.000700,-0.000374
3,0.001304,0.001528,-0.000053,0.012543,0.001020,0.000961,0.001045,-0.000435,0.002855,0.000812,0.002634,-0.000612,0.000330,0.001432,0.000442,0.000063,0.002760,0.000948,-0.000924,0.002184
4,0.001945,0.001794,-0.000852,0.001020,0.014525,-0.001656,0.001454,-0.001324,-0.000028,-0.000170,0.000529,0.000088,-0.000122,-0.000097,-0.000385,-0.000529,-0.000042,0.000889,-0.000345,-0.000043


In [101]:
L_inv, _ = dtrtri(L, lower=True)

A_inv_4 = L_inv.T @ L_inv
display(pd.DataFrame(A_inv_4).head()) 

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.012248,0.000140,-0.001994,0.001304,0.001945,-0.001190,-0.000150,-0.001785,0.001686,0.000066,0.001683,-0.000680,-0.002132,-0.000263,0.001831,-0.002084,0.001304,-0.000197,0.000847,0.001176
1,0.000140,0.012355,0.000723,0.001528,0.001794,0.001394,0.000892,-0.002599,0.001468,-0.001099,0.001164,0.000180,0.001735,0.000687,0.000266,-0.000304,0.001885,-0.002428,-0.001702,0.001074
2,-0.001994,0.000723,0.012951,-0.000053,-0.000852,0.001067,0.001231,0.001516,0.000820,0.000027,-0.001144,0.000260,0.000917,0.001398,0.000488,0.002026,0.000368,0.002173,0.000700,-0.000374
3,0.001304,0.001528,-0.000053,0.012543,0.001020,0.000961,0.001045,-0.000435,0.002855,0.000812,0.002634,-0.000612,0.000330,0.001432,0.000442,0.000063,0.002760,0.000948,-0.000924,0.002184
4,0.001945,0.001794,-0.000852,0.001020,0.014525,-0.001656,0.001454,-0.001324,-0.000028,-0.000170,0.000529,0.000088,-0.000122,-0.000097,-0.000385,-0.000529,-0.000042,0.000889,-0.000345,-0.000043


In [111]:
A_invs = [A_inv, A_inv_2, A_inv_3, A_inv_4]
A_invs_name = ["A_inv", "A_inv_2", "A_inv_3", "A_inv_4"]

df = pd.DataFrame(columns=range(4), index=range(4))

for i in range(len(A_invs)):
    for j in range(len(A_invs)):
        df.iloc[i, j] = np.unique(A_invs[i] == A_invs[j])

df.columns, df.index = A_invs_name, A_invs_name
display(df)

,A_inv,A_inv_2,A_inv_3,A_inv_4
A_inv,[True],"[False, True]","[False, True]","[False, True]"
A_inv_2,"[False, True]",[True],"[False, True]","[False, True]"
A_inv_3,"[False, True]","[False, True]",[True],"[False, True]"
A_inv_4,"[False, True]","[False, True]","[False, True]",[True]


In [112]:
threshold = 1e-13

df2 = pd.DataFrame(columns=range(4), index=range(4))

for i in range(len(A_invs)):
    for j in range(len(A_invs)):
        df2.iloc[i, j] = np.sum(np.abs(A_invs[i] - A_invs[j]) > threshold)

df2.columns, df2.index = A_invs_name, A_invs_name
display(df2)

,A_inv,A_inv_2,A_inv_3,A_inv_4
A_inv,0,0,0,0
A_inv_2,0,0,0,0
A_inv_3,0,0,0,0
A_inv_4,0,0,0,0


## 2.3 np.ascontiguousarray

# パフォーマンス

## 2.4 np.asfortranarray